In [5]:
import numpy as np
import pickle
from memory_profiler import memory_usage
import time
import heapq
import math
import tracemalloc
import time
import gc
import csv


In [6]:

# General Notes:
# - Update the provided file name (code_<RollNumber>.py) as per the instructions.
# - Do not change the function name, number of parameters or the sequence of parameters.
# - The expected output for each function is a path (list of node names)
# - Ensure that the returned path includes both the start node and the goal node, in the correct order.
# - If no valid path exists between the start and goal nodes, the function should return None.

#intializing global variables
global path_exists
path_exists = None


class Node:
    def __init__(self, state, h_score=0, parent=None, depth=0, path_cost=0, x=0, y=0, f_score=0):
        self.state = state
        self.parent = parent
        self.depth = depth
        self.path_cost = path_cost

        # Used only in case of A* algorithms
        self.x = x
        self.y = y
        self.h_score = h_score
        self.f_score = f_score

    def expand(self, adj_matrix, rev=False):
        # Expands a node and allocates memory for each child node
        children = []
        node = self.state
        for child, cost in enumerate(adj_matrix[node]):
            if cost != 0:
                children.append(Node(child, parent=self, depth=self.depth + 1, path_cost=self.path_cost + cost))
        if rev:
            children.reverse()
        return children


    def __lt__(self, other):
        # Used in the A* algorithm, for inserting the nodes in a priority queue based on heuristic
        return self.f_score < other.f_score





def floyd_warshall(adj_matrix):
    n = len(adj_matrix)

    # Initialize a matrix for path existence (True if path exists, False otherwise)
    path_matrix = np.array(adj_matrix, dtype=bool)

    # Floyd-Warshall algorithm
    for k in range(n):
        for i in range(n):
            for j in range(n):
                path_matrix[i][j] = path_matrix[i][j] or (path_matrix[i][k] and path_matrix[k][j])

    return path_matrix


def in_cycle(node):
    ancestor_node = node.parent
    while ancestor_node is not None:
        if node.state == ancestor_node.state:
            return True
        ancestor_node = ancestor_node.parent
    return False

def find_path(node):
    path = []
    while node is not None:
        path.append(node.state)
        node = node.parent
    path.reverse()
    return path

def dfs(adj_matrix, start_node, goal_node, depth):
    frontier = [Node(start_node)]
    cutoff_occurred = False

    while frontier:
        node = frontier.pop()

        if node.state == goal_node:
            return node

        if node.depth > depth:
            cutoff_occurred = True
        else:
            for child in node.expand(adj_matrix, rev=True):
                if not in_cycle(child):
                    frontier.append(child)

    return 'cutoff' if cutoff_occurred else None

def get_ids_path(adj_matrix, start_node, goal_node):
    if not path_exists[start_node][goal_node]:
        return None

    depth = 0
    max_depth = len(adj_matrix)

    while depth < max_depth:
        result = dfs(adj_matrix, start_node, goal_node, depth)

        if result == 'cutoff':
            depth += 1
        elif result is None:
            return None
        else:
            return find_path(result)

    return None





# Algorithm: Bi-Directional Search

# Input:
#   - adj_matrix: Adjacency matrix representing the graph.
#   - start_node: The starting node in the graph.
#   - goal_node: The target node in the graph.

# Return:
#   - A list of node names representing the path from the start_node to the goal_node.
#   - If no path exists, the function should return None.

# Sample Test Cases:

#   Test Case 1:
#     - Start node: 1, Goal node: 2
#     - Return: [1, 7, 6, 2]

#   Test Case 2:
#     - Start node: 5, Goal node: 12
#     - Return: [5, 97, 98, 12]

#   Test Case 3:
#     - Start node: 12, Goal node: 49
#     - Return: None

#   Test Case 4:
#     - Start node: 4, Goal node: 12
#     - Return: [4, 6, 2, 9, 8, 5, 97, 98, 12]



def join_nodes(node_f, node_b):
    # Finds the path given the forward frontier and backward frontier
    path = []
    while node_f:
        path.append(node_f.state)
        node_f = node_f.parent
    path.reverse()

    # Collect the backward path
    node_b_path = []
    while node_b:
        node_b_path.append(node_b.state)
        node_b = node_b.parent

    # Combine paths
    path.extend(node_b_path)
    return path

def get_bidirectional_search_path(adj_matrix, start_node, goal_node):
    # Initialization
    node_f = Node(start_node)
    node_b = Node(goal_node)
    frontier_f = [node_f]
    frontier_b = [node_b]
    visited_f = {node_f.state: node_f}
    visited_b = {node_b.state: node_b}


    while frontier_f and frontier_b:
        # Forward search
        nf = frontier_f.pop(0)
        if nf.state in visited_b:
            return join_nodes(nf, visited_b[nf.state])

        for child in nf.expand(adj_matrix):
            if child.state not in visited_f:
                visited_f[child.state] = child
                frontier_f.append(child)

        # Backward search
        nb = frontier_b.pop(0)
        if nb.state in visited_f:
            return join_nodes(visited_f[nb.state], nb)

        for child in nb.expand(adj_matrix):
            if child.state not in visited_b:
                visited_b[child.state] = child
                frontier_b.append(child)

    gc.collect()

    return None




In [7]:


def measure_time_1(algorithm, adj_matrix, start_node, goal_node):

    start_time = time.time()  # Start time
    path = algorithm(adj_matrix, start_node, goal_node)
    end_time = time.time() # End time

    print(f"Algorithm: {algorithm.__name__}")
    print(f"Time taken: {end_time - start_time} seconds")

    return end_time - start_time



def measure_memory_1(algorithm, adj_matrix, start_node, goal_node):

    tracemalloc.start()  # Start tracing memory allocation
    path = algorithm(adj_matrix, start_node, goal_node)
    current_mem, peak_mem = tracemalloc.get_traced_memory()  # Get current memory usage
    tracemalloc.stop()  # Stop tracing memory

    current_mem_mb = current_mem / 10**6  # Convert to MB

    print(f"Algorithm: {algorithm.__name__}")
    print(f"Current memory usage: {current_mem_mb} MB")

    return current_mem_mb


In [8]:

adj_matrix = np.load('IIIT_Delhi.npy')
with open('IIIT_Delhi.pkl', 'rb') as f:
    node_attributes = pickle.load(f)

global path_exists

path_exists = floyd_warshall(adj_matrix)

# Initialize lists to store metrics for each iteration
ids_times = []
ids_memories = []
bbfs_times = []
bbfs_memories = []

# Initialize total time and memory
ids_time_total = 0
ids_memory_total = 0
bbfs_time_total = 0
bbfs_memory_total = 0

# Iteration counter
iteration = 1

# Loop to measure time and memory for each iteration
for i in range(35):
    for j in range(35):
        print("Iteration: ", iteration)
        
        # Measure time and memory for IDS
        ids_time = measure_time_1(get_ids_path, adj_matrix, i, j)
        ids_memory = measure_memory_1(get_ids_path, adj_matrix, i, j)
        ids_times.append(ids_time)
        ids_memories.append(ids_memory)
        ids_time_total += ids_time
        ids_memory_total += ids_memory
        
        # Measure time and memory for BBFS
        bbfs_time = measure_time_1(get_bidirectional_search_path, adj_matrix, i, j)
        bbfs_memory = measure_memory_1(get_bidirectional_search_path, adj_matrix, i, j)
        bbfs_times.append(bbfs_time)
        bbfs_memories.append(bbfs_memory)
        bbfs_time_total += bbfs_time
        bbfs_memory_total += bbfs_memory

        iteration += 1

# Write the metrics to a CSV file
with open('uninformed_algorithm_performance.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(['Iteration', 'IDS Time (seconds)', 'IDS Memory (MB)', 'BBFS Time (seconds)', 'BBFS Memory (MB)'])
    
    # Write data for each iteration
    for iter_num in range(len(ids_times)):
        writer.writerow([iter_num + 1, ids_times[iter_num], ids_memories[iter_num], bbfs_times[iter_num], bbfs_memories[iter_num]])

# Print total time and memory for each algorithm
print(f"Total Iterative Deepening Search time: {ids_time_total:.6f} seconds")
print(f"Total Iterative Deepening Search memory: {ids_memory_total:.6f} MB")
print(f"Total Bidirectional Heuristic Time: {bbfs_time_total:.6f} seconds")
print(f"Total Bidirectional Heuristic memory: {bbfs_memory_total:.6f} MB")



Iteration:  1
Algorithm: get_ids_path
Time taken: 9.298324584960938e-06 seconds
Algorithm: get_ids_path
Current memory usage: 5.758186 MB
Algorithm: get_bidirectional_search_path
Time taken: 3.743171691894531e-05 seconds
Algorithm: get_bidirectional_search_path
Current memory usage: 3.2e-05 MB
Iteration:  2
Algorithm: get_ids_path
Time taken: 6.4373016357421875e-06 seconds
Algorithm: get_ids_path
Current memory usage: 0.0 MB
Algorithm: get_bidirectional_search_path
Time taken: 0.04767560958862305 seconds
Algorithm: get_bidirectional_search_path
Current memory usage: 6.4e-05 MB
Iteration:  3
Algorithm: get_ids_path
Time taken: 9.059906005859375e-06 seconds
Algorithm: get_ids_path
Current memory usage: 0.0 MB
Algorithm: get_bidirectional_search_path
Time taken: 0.043955087661743164 seconds
Algorithm: get_bidirectional_search_path
Current memory usage: 6.4e-05 MB
Iteration:  4
Algorithm: get_ids_path
Time taken: 1.4543533325195312e-05 seconds
Algorithm: get_ids_path
Current memory usage: 